 **Collaborative Filtering Based on Item Similarity**

---



## Inisiasi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [ ]:
pip install scikit-surprise

     |████████████████████████████████| 11.8MB 241kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617660 sha256=675f001e4e2b47fa36142522c757fd66712cdde674c760204915227d87df8d89
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
pip install turicreate

     |████████████████████████████████| 92.0MB 1.4MB/s 
     |████████████████████████████████| 327kB 47.1MB/s 
     |████████████████████████████████| 3.5MB 26.2MB/s 
     |████████████████████████████████| 3.6MB 17.1MB/s 
     |████████████████████████████████| 86.4MB 36kB/s 
     |████████████████████████████████| 18.3MB 172kB/s 
     |████████████████████████████████| 3.8MB 20.2MB/s 
     |████████████████████████████████| 450kB 46.1MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 2.9MB 34.2MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-cp37-none-any.whl size=13716 sha256=209cd756b812ca3d20492e58d4c253a13049e7414d151f0e2352b4dbebe95938
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for resampy: filename=resampy-0.2.1-cp37-none-any.whl size=320858 sha256=beb154d0a759c19c2f3703c34334876d0238fbfbe7691cadeba7b7d85663b735
  Stored in directory

In [ ]:
from surprise import Dataset
from surprise import Reader
import turicreate as tc

#Download dataset

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Dataset-WebsiteGlobo/articles_metadata.csv.zip', 'r')
zip_ref.extractall('files')

In [ ]:
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Dataset-WebsiteGlobo/clicks.zip', 'r')
zip_ref.extractall('files')

In [ ]:
articles =  pd.read_csv('/content/files/articles_metadata.csv')
articles

,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162
...,...,...,...,...,...
364042,364042,460,1434034118000,0,144
364043,364043,460,1434148472000,0,463
364044,364044,460,1457974279000,0,177
364045,364045,460,1515964737000,0,126


#Ambil fitur

In [ ]:
articles.drop(['created_at_ts', 'publisher_id', 'words_count'], inplace=True, axis=1)
articles

,article_id,category_id
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
364042,364042,460
364043,364043,460
364044,364044,460
364045,364045,460


In [ ]:
# y = pd.get_dummies(articles.category_id, prefix='category')
# y

In [ ]:
# articles.drop(['category_id'], inplace=True, axis=1)
# articles = articles.join(y)
# articles

In [ ]:
clickspath = glob.glob('/content/files/clicks/*.csv')
clicks = pd.concat([pd.read_csv(f) for f in clickspath ])
clicks

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,29403,1508099783178606,1508099783000,3,202355,1508099806928,4,3,2,1,7,2
1,29403,1508099783178606,1508099783000,3,209236,1508102151101,4,3,2,1,7,2
2,29403,1508099783178606,1508099783000,3,202559,1508102181101,4,3,2,1,7,2
3,296441,1508099784231607,1508099784000,2,209236,1508099784551,4,3,2,1,11,2
4,296441,1508099784231607,1508099784000,2,202355,1508099814551,4,3,2,1,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...
4082,180706,1507476974381117,1507476974000,3,42329,1507477275948,4,1,12,1,21,7
4083,219065,1507476976121118,1507476976000,2,129434,1507477369570,4,3,2,1,20,2
4084,219065,1507476976121118,1507476976000,2,198199,1507477399570,4,3,2,1,20,2
4085,230894,1507476976165119,1507476976000,2,348113,1507476999119,4,1,17,1,9,2


In [ ]:
clicks.drop(['session_id', 'session_start', 'session_size', 'click_timestamp', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type'], inplace=True, axis=1)
clicks = clicks.rename(columns={'click_article_id':'article_id'})
clicks

,user_id,article_id
0,29403,202355
1,29403,209236
2,29403,202559
3,296441,209236
4,296441,202355
...,...,...
4082,180706,42329
4083,219065,129434
4084,219065,198199
4085,230894,348113


#Merge Dataframe

In [ ]:
clicks = pd.merge(clicks, articles, how='left', on=['article_id'])

In [ ]:
clicks

,user_id,article_id,category_id
0,29403,202355,327
1,29403,209236,332
2,29403,202559,327
3,296441,209236,332
4,296441,202355,327
...,...,...,...
2988176,180706,42329,67
2988177,219065,129434,252
2988178,219065,198199,323
2988179,230894,348113,442


#Hapus user yang baca artikel <5

In [ ]:
a = clicks[['user_id', 'category_id']].groupby(['user_id']).count()
a = a.reset_index()
a = a.rename(columns={'category_id':'count'})
a

,user_id,count
0,0,8
1,1,12
2,2,4
3,3,17
4,4,7
...,...,...
322892,322892,2
322893,322893,2
322894,322894,2
322895,322895,2


In [ ]:
pd.options.display.max_rows = 999
listt = a.groupby('count').count()
print(listt)
pd.options.display.max_rows = 10

       user_id
count         
2       101428
3        22660
4        38432
5        18070
6        20370
7        13358
8        12780
9         9701
10        8807
11        7143
12        6485
13        5500
14        4990
15        4284
16        3787
17        3458
18        3040
19        2755
20        2557
21        2281
22        2120
23        1864
24        1725
25        1554
26        1487
27        1382
28        1282
29        1129
30        1005
31         997
32         933
33         822
34         784
35         663
36         706
37         648
38         594
39         555
40         531
41         514
42         443
43         439
44         419
45         397
46         360
47         334
48         344
49         331
50         284
51         280
52         240
53         220
54         246
55         200
56         194
57         190
58         183
59         187
60         193
61         163
62         171
63         146
64         134
65         141
66        

In [ ]:
listt

,user_id
count,
2,101428
3,22660
4,38432
5,18070
6,20370
...,...
746,1
783,1
900,1


In [ ]:
a = a.loc[a['count']>=5].reset_index(drop=True)
a

,user_id,count
0,0,8
1,1,12
2,3,17
3,4,7
4,5,87
...,...,...
160372,322707,5
160373,322710,7
160374,322712,5
160375,322732,5


In [ ]:
clicks = clicks.loc[clicks['user_id'].isin(a['user_id'])]
clicks

,user_id,article_id,category_id
0,29403,202355,327
1,29403,209236,332
2,29403,202559,327
5,30738,209236,332
6,30738,202355,327
...,...,...,...
2988176,180706,42329,67
2988177,219065,129434,252
2988178,219065,198199,323
2988179,230894,348113,442


#Inisiasi model

In [ ]:
articles = tc.SFrame(data=articles)

In [ ]:
clicks = tc.SFrame(data=clicks)

In [ ]:
articles

article_id,category_id
0,0
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [ ]:
clicks

user_id,article_id,category_id
29403,202355,327
29403,209236,332
29403,202559,327
30738,209236,332
30738,202355,327
161431,202355,327
161431,209236,332
23622,202355,327
23622,209236,332
23622,218028,352


In [ ]:
clicks['category_id'] = clicks['category_id'].astype(str)
clicks

user_id,article_id,category_id
29403,202355,327
29403,209236,332
29403,202559,327
30738,209236,332
30738,202355,327
161431,202355,327
161431,209236,332
23622,202355,327
23622,209236,332
23622,218028,352


#Split data

In [ ]:
training_data, validation_data = tc.recommender.util.random_split_by_user(clicks, 'user_id', 'article_id')

#Bikin model

In [ ]:
model = tc.ranking_factorization_recommender.create(training_data, 'user_id', 'article_id')

Preparing data set.

Data has 2560220 observations with 160377 users and 37869 items.

Data prepared in: 3.36885s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.04401

Final training Predictive Error: 0.00882098

In [ ]:
model_recomm = model.recommend()

recommendations finished on 1000/160377 queries. users per second: 285.769

recommendations finished on 2000/160377 queries. users per second: 288.371

recommendations finished on 3000/160377 queries. users per second: 288.742

recommendations finished on 4000/160377 queries. users per second: 289.523

recommendations finished on 5000/160377 queries. users per second: 289.847

...

recommendations finished on 156000/160377 queries. users per second: 289.528

recommendations finished on 157000/160377 queries. users per second: 289.531

recommendations finished on 158000/160377 queries. users per second: 289.531

recommendations finished on 159000/160377 queries. users per second: 289.532

recommendations finished on 160000/160377 queries. users per second: 289.536

In [ ]:
item_sim_model = tc.item_similarity_recommender.create(training_data, user_id='user_id', item_id='article_id', similarity_type='cosine')

Warning: Ignoring columns category_id;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 2560220 observations with 160377 users and 37869 items.

...

| 28.26s                              | 100              | 37869           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 29.4421s

In [ ]:
item_sim_recomm = item_sim_model.recommend()

recommendations finished on 1000/160377 queries. users per second: 3128.15

recommendations finished on 2000/160377 queries. users per second: 3195.8

recommendations finished on 3000/160377 queries. users per second: 3214.64

recommendations finished on 4000/160377 queries. users per second: 3190.69

recommendations finished on 5000/160377 queries. users per second: 3195.05

...

recommendations finished on 156000/160377 queries. users per second: 3229.42

recommendations finished on 157000/160377 queries. users per second: 3229.67

recommendations finished on 158000/160377 queries. users per second: 3230.07

recommendations finished on 159000/160377 queries. users per second: 3228.34

recommendations finished on 160000/160377 queries. users per second: 3226.36

#Bandingin model

In [ ]:
model.evaluate_precision_recall(validation_data)

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 15714
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |  99274  |   1    |    0.0    |  0.0   |   3   |
 |  99274  |   2    |    0.0    |  0.0   |   3   |
 |  99274  |   3    |    0.0    |  0.0   |   3   |
 |  99274  |   4    |    0.0    |  0.0   |   3   |
 |  99274  |   5    |    0.0    |  0.0   |   3   |
 |  99274  |   6    |    0.0    |  0.0   |   3   |
 |  99274  |   7    |    0.0    |  0.0   |   3   |
 |  99274  |   8    |    0.0    |  0.0   |   3   |
 |  99274  |   9    |    0.0    |  0.0   |   3   |
 |  99274  |   10   |    0.0    |  0.0   |   3   |
 +---------+--------+-----------+--------+-------+
 [15714 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

In [ ]:
item_sim_model.evaluate_precision_recall(validation_data)

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 15732
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |  206854 |   1    |    0.0    |  0.0   |   4   |
 |  206854 |   2    |    0.0    |  0.0   |   4   |
 |  206854 |   3    |    0.0    |  0.0   |   4   |
 |  206854 |   4    |    0.0    |  0.0   |   4   |
 |  206854 |   5    |    0.0    |  0.0   |   4   |
 |  206854 |   6    |    0.0    |  0.0   |   4   |
 |  206854 |   7    |    0.0    |  0.0   |   4   |
 |  206854 |   8    |    0.0    |  0.0   |   4   |
 |  206854 |   9    |    0.0    |  0.0   |   4   |
 |  206854 |   10   |    0.0    |  0.0   |   4   |
 +---------+--------+-----------+--------+-------+
 [15732 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

In [ ]:
model_performance = tc.recommender.util.compare_models = (validation_data, [model, item_sim_model])
print (model_performance)

(Columns:
	user_id	int
	article_id	int
	category_id	str

Rows: 3397

Data:
+---------+------------+-------------+
| user_id | article_id | category_id |
+---------+------------+-------------+
|  46924  |   209236   |     332     |
|  26703  |   331116   |     436     |
|  26703  |   315146   |     431     |
|  31842  |   202355   |     327     |
|  31842  |   202559   |     327     |
|  20410  |   331116   |     436     |
|  26110  |   168761   |     297     |
|  36687  |   183716   |     301     |
|  233069 |   195881   |     317     |
|  22894  |   218028   |     352     |
+---------+------------+-------------+
[3397 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., [Class                            : RankingFactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : article_id
Target                           : None
Additional obs

# Rekomendasi

In [ ]:
%time
item_sim_recomm

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


user_id,article_id,score,rank
29403,272143,0.0713270256916682,1
29403,331116,0.06542522211869557,2
29403,272660,0.06400906046231587,3
29403,209013,0.06085356076558431,4
29403,206785,0.0567471981048584,5
29403,160974,0.05438982446988424,6
29403,206763,0.05212643245855967,7
29403,156560,0.051204209526379905,8
29403,207603,0.05066779752572378,9
29403,209348,0.05025159319241842,10


In [ ]:
%time
item_sim_recomm[item_sim_recomm['user_id']==20001]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


user_id,article_id,score,rank
20001,59057,0.03902427418522103,1
20001,96210,0.038132808826587816,2
20001,234698,0.03691431170418149,3
20001,64329,0.03305111044929141,4
20001,183176,0.032149589250958156,5
20001,95716,0.029911159838318192,6
20001,129434,0.029233889289633937,7
20001,336220,0.02905504192624773,8
20001,289090,0.02880396161760603,9
20001,235616,0.02755046805376729,10


In [ ]:
%time
item_sim_recomm[item_sim_recomm['user_id']==20001][0]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.34 µs


{'article_id': 59057,
 'rank': 1,
 'score': 0.03902427418522103,
 'user_id': 20001}

In [ ]:
%time
item_sim_recomm[item_sim_recomm['user_id']==20001][0]['article_id']

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


59057

In [ ]:
item_sim_recomm[item_sim_recomm['article_id']==59057]

user_id,article_id,score,rank
72919,59057,0.12323407693342729,3
35561,59057,0.05507513880729675,10
59476,59057,0.07848335305849712,7
214903,59057,0.08370636518184955,10
51039,59057,0.07465610366601211,7
141703,59057,0.10111986215297993,4
15688,59057,0.08630590335182521,5
24363,59057,0.04202838313011896,9
177717,59057,0.060849279165267944,10
40408,59057,0.08460032277637058,6


In [ ]:
clicks[clicks['user_id']==20001]

user_id,article_id,category_id
20001,292508,421
20001,123569,250
20001,331116,436
20001,285101,412
20001,257291,391
20001,282875,412
20001,158646,281
20001,314356,431
20001,118712,247
20001,313137,431


In [ ]:
%time clicks[clicks['article_id']==59057]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


user_id,article_id,category_id
64638,59057,118
303634,59057,118
43316,59057,118
291452,59057,118
92577,59057,118
138472,59057,118
37783,59057,118
201,59057,118
1238,59057,118
102415,59057,118


In [ ]:
item_sim_model.save("/content/drive/MyDrive/Dataset-WebsiteGlobo/item_sim_model.model")

In [ ]:
item_sim_model = tc.load_model("/content/drive/MyDrive/Dataset-WebsiteGlobo/item_sim_model.model")

In [ ]:
%time new_obs_data = tc.SFrame({'user_id' : [20000], 'article_id' : [59057]})
%time recommendations = item_sim_model.recommend([20000], new_observation_data = new_obs_data)
%time recommendations

CPU times: user 664 µs, sys: 0 ns, total: 664 µs
Wall time: 839 µs
CPU times: user 5.05 ms, sys: 2.98 ms, total: 8.03 ms
Wall time: 12.8 ms
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


user_id,article_id,score,rank
20000,336223,0.28627830743789673,1
20000,133160,0.28260546922683716,2
20000,288320,0.2773144245147705,3
20000,270229,0.24532932043075562,4
20000,235870,0.22432857751846313,5
20000,303180,0.21951007843017578,6
20000,48403,0.21499228477478027,7
20000,119193,0.21140241622924805,8
20000,234698,0.20978236198425293,9
20000,141004,0.2073245644569397,10


In [ ]:
new_obs_data = tc.SFrame({'user_id' : [20000, 20000],
                                'article_id' : [59057, 272143]})
recommendations = item_sim_model.recommend([20000], new_observation_data = new_obs_data)
recommendations

user_id,article_id,score,rank
20000,199198,0.27361056208610535,1
20000,336223,0.2566239833831787,2
20000,198659,0.2478254735469818,3
20000,64329,0.24696749448776245,4
20000,133160,0.22180059552192688,5
20000,336221,0.21919095516204834,6
20000,272660,0.21032017469406128,7
20000,288320,0.20658227801322937,8
20000,234698,0.2051093578338623,9
20000,235870,0.2033531665802002,10


In [ ]:
new_obs_data = tc.SFrame({'user_id' : [20001, 20001],
                                'article_id' : [59057, 96210]})
recommendations = item_sim_model.recommend([20001], new_observation_data = new_obs_data)
recommendations

user_id,article_id,score,rank
20001,234698,0.038497173973403055,1
20001,64329,0.03428200963904096,2
20001,183176,0.033571429277589805,3
20001,95716,0.03155771662427493,4
20001,129434,0.030574961482542348,5
20001,336220,0.030567807988970692,6
20001,289090,0.03017680807263439,7
20001,235616,0.02872895195846158,8
20001,128260,0.027764933895690277,9
20001,235870,0.027154989891651413,10


In [ ]:
new_obs_data = tc.SFrame({'user_id' : [20001],
                                'article_id' : [59057]})
recommendations = item_sim_model.recommend([20001], new_observation_data = new_obs_data)
recommendations

user_id,article_id,score,rank
20001,96210,0.038775370622936046,1
20001,234698,0.03782414354776081,2
20001,64329,0.033619478501771626,3
20001,183176,0.03269593464700799,4
20001,95716,0.030747218194760773,5
20001,129434,0.02979397083583631,6
20001,336220,0.029759943798968666,7
20001,289090,0.02946980783813878,8
20001,235616,0.028163839013952956,9
20001,128260,0.02706392909351148,10


In [ ]:
item_data = tc.SFrame({"my_item_id" : ['a','b','c','d'],
                           "data_1" : [ [1, 0,1], [1, 0,4], [0, 1,6], [0.5, 0.5,0.5] ],
                           "data_2" : [ [0, 1,2], [1, 0,3], [0, 1,7], [0.5, 0.5,1] ] })
item_data

data_1,data_2,my_item_id
"[1.0, 0.0, 1.0]","[0.0, 1.0, 2.0]",a
"[1.0, 0.0, 4.0]","[1.0, 0.0, 3.0]",b
"[0.0, 1.0, 6.0]","[0.0, 1.0, 7.0]",c
"[0.5, 0.5, 0.5]","[0.5, 0.5, 1.0]",d
